In [ ]:
stopwords = list(set([
    '이', '가', '은', '는', '을', '를', '의', '에', '에서', '에게', '께', '로', '으로', '하다', '있다',
    '와', '과', '보다', '처럼', '만큼', '같이', '까지', '마저', '조차', '부터', 
    '이나', '나', '이며', '며', '등', '하다', '한다', '하고', '하니', '하면', 
    '되어', '되다', '되고', '되니', '입니다', '습니다', 'ㅂ니다', '어요', '아요', '다', '방이', '제대로',
    '고', '면', '게', '지', '죠',
    '그리고', '그러나', '하지만', '그런데', '그래서', '그러면', '그러므로', '따라서', 
    '또한', '또는', '및', '즉', '한편', '반면에', '근데',
    '나', '저', '우리', '저희', '너', '너희', '당신', '그', '그녀', '그들', '누구', '그렇다',
    '무엇', '어디', '언제', '어느', '이것', '그것', '저것', '여기', '거기', '저기', 
    '이쪽', '그쪽', '저쪽',
    '하나', '둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '열',
    '일', '이', '삼', '사', '오', '육', '칠', '팔', '구', '십', '백', '천', '만',
    '첫째', '둘째', '셋째',
    '바로', '때', '것', '수', '문제', '경우', '부분', '이다',
    '내용', '결과', '자체', '가지', '있다',
    '않았어요', '있었어요', '했어요', '했는데요', '있는데요', '합니다', '없다', '나다','생각하다',
    '했다', '같다', '네요','아니다',
    '좀', '너무', '정말', '많이', '조금',
    '사장', '이용', '용하다', '물이', '매우',
    '뿐', '대로', '만', '따름', '나름', '김에', '터',
    '아', '아이고', '아이구', '아하', '어', '그래', '응', '네', '예', '아니', '않다', '안되다','안','그냥',
    '가다', '오다', '주다', '말다', '나다', '받다', '알다', '모르다', '싶다', '생각하다', '들다'
]))

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset
import pandas as pd
import torch

# 1. 데이터 로드
df = pd.read_csv("keyword_test.csv")  # 'clean_reviews'와 'label' 컬럼이 있어야 함
df = df.dropna(subset=['clean_reviews', 'label'])  # 결측 제거
df['label'] = df['label'].astype(int)  # 레이블 정수형 변환

# 2. 데이터셋 분할
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# 3. 토크나이저 로드 (KoBERT)
model_name = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 4. 토큰화 함수 정의
def tokenize_function(example):
    return tokenizer(example['clean_reviews'], padding="max_length", truncation=True, max_length=128)

# 5. HuggingFace Dataset으로 변환
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# 6. 토큰화 적용
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# 7. label, input_ids, attention_mask만 추림
train_dataset = train_dataset.remove_columns(['clean_reviews'])
val_dataset = val_dataset.remove_columns(['clean_reviews'])

# 8. PyTorch tensor 형식 명시
train_dataset.set_format("torch")
val_dataset.set_format("torch")

# 9. 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 10. 평가지표 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds, average='weighted')
    }

# 11. 학습 설정
training_args = TrainingArguments(
    output_dir="./bert_sentiment_output",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=1,
    seed=42
)

# 12. Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# 13. 학습 실행
trainer.train()

# 14. 모델 저장
trainer.save_model("./saved_bert_sentiment_model")
tokenizer.save_pretrained("./saved_bert_sentiment_model")
